In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import shap

In [12]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split

n_features = 20 #20,50,100,250,500
n_informative = int(0.33*n_features) #5%,10%,33%,50%,90%
n_samples = int(5000/(1-0.33))+1 #7463#5000

X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=n_informative, n_redundant=0, n_repeated = 0,shuffle=False)
#X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative,random_state=4,shuffle=False)
X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=42)
X_train

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19
4529,-2.507711,-1.380179,-1.731852,2.759644,-1.648487,-0.094561,-0.414940,-0.517523,0.652992,0.166092,0.937923,0.068159,0.698941,-0.494359,-0.926061,-0.307777,0.306507,0.179015,0.282898,-0.128178
2897,-3.165850,-0.455225,-1.546249,-3.717344,0.150185,2.324727,1.542446,-1.141088,0.542797,-0.405756,1.332337,-1.655105,0.556727,1.519054,-0.140718,-0.255736,0.255354,1.707274,-1.867323,0.278887
6361,-0.523556,3.138877,-1.062066,2.906435,2.927918,-1.109569,0.538517,-0.348787,1.112268,-0.300068,-0.294329,-0.028526,1.271090,-0.168948,0.122677,0.717923,0.975971,0.555093,0.908571,-0.269678
1654,-1.137635,2.983856,0.112493,1.364167,-1.985387,-2.438406,-0.133302,-1.299570,-0.680647,-1.369259,-1.373810,0.091342,-1.197139,0.233148,-0.047254,-0.872664,-0.535044,-2.670622,-0.077298,1.637037
6111,-1.975512,-0.143283,-0.386317,-0.201906,0.964193,-3.191712,-0.027806,0.622075,-0.444076,2.199865,-0.109465,0.760795,0.097191,0.138494,-0.960107,1.513340,0.414545,0.121863,2.852718,-0.499749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,-1.766048,-1.171273,-1.416255,4.246000,-1.311990,1.760119,-0.012331,-0.229009,1.608737,-0.218211,0.721106,0.642894,-0.133245,-0.380248,0.180807,1.067949,0.600325,0.222074,-1.084148,-1.538107
5226,-1.566735,0.378162,0.404641,-1.911543,2.181362,-4.116653,-0.623058,0.009050,0.520340,0.907352,1.095978,0.605265,0.496711,-1.235390,0.484506,0.873014,-0.147343,0.570402,-0.133447,0.370973
5390,-2.557793,-2.536380,-3.467047,-0.065625,-1.879515,-1.965617,0.016775,-1.222711,-1.577725,-0.283389,0.216007,-2.001660,1.271304,1.007134,1.119911,0.081895,0.811402,0.645666,1.283420,-2.663874
860,1.902300,2.428416,-0.101549,1.859264,-1.199220,-2.104229,-0.154633,-0.895444,-0.797047,-0.927523,0.986585,0.735307,0.421717,-0.183612,-0.965146,-0.813010,0.824310,-1.004212,0.017964,-2.011507


In [13]:
import shapicant 
from catboost import CatBoostClassifier,CatBoostRegressor

X, y = make_classification(n_samples=5000, n_classes=2, n_features=n_features, n_informative=n_informative, n_redundant=0, n_repeated = 0,shuffle=False)
#X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative,random_state=4,shuffle=False)
X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

Inp_db = X.copy(deep=True)
Inp_db["class"]=y
Inp_db = Inp_db.reset_index()
Index_col = "index"
target_col = "class"

train_idx,val_idx = train_test_split(Inp_db["index"],test_size=0.2,random_state = 0)

X_train = Inp_db[Inp_db[Index_col].isin(train_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
X_val = Inp_db[Inp_db[Index_col].isin(val_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
Y_train = Inp_db[Inp_db[Index_col].isin(train_idx)][target_col]

# LightGBM in RandomForest-like mode (with rows subsampling), without columns subsampling
model = CatBoostClassifier(verbose=False,iterations=250,use_best_model=False)

# This is the class (not its instance) of SHAP's TreeExplainer
explainer_type = shap.TreeExplainer

# Use PandasSelector with 100 iterations
selector = shapicant.PandasSelector(model, explainer_type, random_state=42)

# Run the feature selection
# If we provide a validation set, SHAP values are computed on it, otherwise they are computed on the training set
# We can also provide additional parameters to the underlying estimator's fit method through estimator_params
selector.fit(X_train, Y_train, X_validation=X_val)#, estimator_params={"categorical_feature": None})

# Just get the features list
selected_features = selector.get_features()

# We can also get the p-values as pandas Series
p_values = selector.p_values_

np.array(selected_features)


Computing null SHAP values: 100%|████████████████████████████████████████████████████| 100/100 [01:59<00:00,  1.20s/it]


array(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6',
       'col_16'], dtype='<U6')

In [18]:
X, y = make_classification(n_samples=5000, n_classes=2, n_features=n_features, n_informative=n_informative, n_redundant=0, n_repeated = 0,shuffle=False)
X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
X["class"]=y
X = X.reset_index()


explainer_type = shap.TreeExplainer
# if classification is False it is a Regression problem
model = CatBoostClassifier(verbose=0, n_estimators=250,use_best_model=False)
selector = shapicant.PandasSelector(model, explainer_type, random_state=42)

train_idx,val_idx = train_test_split(X["index"].values,test_size=0.2,random_state = 0)

X_train = X[X["index"].isin(train_idx)].copy(deep=True)[list(X.columns.values[1:-1])]
X_val = X[X["index"].isin(val_idx)].copy(deep=True)[list(X.columns.values[1:-1])]
Y_train =  X[X["index"].isin(train_idx)]["class"]

# Run the feature selection
# If we provide a validation set, SHAP values are computed on it, otherwise they are computed on the training set
# We can also provide additional parameters to the underlying estimator's fit method through estimator_params
selector.fit(X_train, Y_train, X_validation=X_val)#, estimator_params={"categorical_feature": None})

subset = selector.get_features()
p_values = selector.p_values_

np.array(subset)


Computing null SHAP values: 100%|████████████████████████████████████████████████████| 100/100 [01:54<00:00,  1.15s/it]


array(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5'], dtype='<U5')

In [16]:
n_informative

6

In [ ]:
import sys
sys.path.append("../powershap")

from powershap import PowerSHAP


from catboost import CatBoostClassifier,CatBoostRegressor
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV,LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier

selector = PowerSHAP(
    model = CatBoostClassifier(verbose=0, n_estimators=250,use_best_model=True),#GradientBoostingClassifier(),#CatBoostClassifier(verbose=0, n_estimators=250),
    #model = CatBoostRegressor(verbose=0, n_estimators=0,use_best_model=True),#GradientBoostingClassifier(),#CatBoostClassifier(verbose=0, n_estimators=250),
    power_iterations=10,automatic=True, limit_automatic=10,verbose=True,force_convergence=True,
)
selector.fit(X_train, y_train)

In [69]:
t = selector._processed_shaps_df
len(t[t.p_value<0.01])

295

In [70]:
np.sort(t[t.p_value<0.01].index.values)

array(['col_0', 'col_1', 'col_101', 'col_102', 'col_104', 'col_105',
       'col_106', 'col_108', 'col_109', 'col_110', 'col_111', 'col_112',
       'col_113', 'col_115', 'col_117', 'col_118', 'col_12', 'col_121',
       'col_122', 'col_123', 'col_125', 'col_126', 'col_127', 'col_13',
       'col_130', 'col_131', 'col_132', 'col_135', 'col_136', 'col_137',
       'col_138', 'col_139', 'col_140', 'col_141', 'col_143', 'col_144',
       'col_145', 'col_146', 'col_147', 'col_150', 'col_153', 'col_155',
       'col_158', 'col_159', 'col_16', 'col_160', 'col_163', 'col_165',
       'col_167', 'col_168', 'col_169', 'col_17', 'col_170', 'col_171',
       'col_172', 'col_173', 'col_175', 'col_18', 'col_183', 'col_184',
       'col_185', 'col_186', 'col_187', 'col_189', 'col_19', 'col_190',
       'col_192', 'col_193', 'col_194', 'col_195', 'col_196', 'col_197',
       'col_2', 'col_20', 'col_200', 'col_201', 'col_203', 'col_204',
       'col_205', 'col_206', 'col_207', 'col_21', 'col_210', 'co

## One by one test

In [ ]:
import sys
sys.path.append("../powershap")

from powershap import PowerSHAP


from catboost import CatBoostClassifier,CatBoostRegressor
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV,LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier

selector = PowerSHAP(
    model = LogisticRegressionCV(max_iter=1000),# CatBoostClassifier(verbose=0, n_estimators=250,use_best_model=True),#GradientBoostingClassifier(),#CatBoostClassifier(verbose=0, n_estimators=250),
    #model = CatBoostRegressor(verbose=0, n_estimators=0,use_best_model=True),#GradientBoostingClassifier(),#CatBoostClassifier(verbose=0, n_estimators=250),
    power_iterations=200,automatic=False, limit_automatic=10,verbose=True
)

In [ ]:
selector_auto = PowerSHAP(
    model = LogisticRegressionCV(max_iter=1000),#CatBoostClassifier(verbose=0, n_estimators=250,use_best_model=True),#GradientBoostingClassifier(),#CatBoostClassifier(verbose=0, n_estimators=250),
    #model = CatBoostRegressor(verbose=0, n_estimators=0,use_best_model=True),#GradientBoostingClassifier(),#CatBoostClassifier(verbose=0, n_estimators=250),
    power_iterations=200,automatic=True, limit_automatic=10,verbose=True
)

In [ ]:
selector.fit(X_train, y_train)
selector_auto.fit(X_train,y_train)

In [ ]:
t = selector_auto._processed_shaps_df
t_inf = np.sum(np.isin(X_train.columns.values[:int(0.9*n_features)],t[t.p_value<0.01].index.values)) 

In [ ]:
import matplotlib.pyplot as plt
from utils import powerSHAP_statistical_analysis

shaps_df = selector._shaps_df
found_features = []
found_informative_features = []
found_noise_features = []

processed_shaps_df = powerSHAP_statistical_analysis(
                        shaps_df[:37],
                        0.01,
                        0.99,
                        include_all=True,
)
processed_shaps_df = processed_shaps_df[processed_shaps_df.index.values!="random_uniform_feature"]

for i in (range(1,100)):
    
    found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<i/100]))
    found_informative_features.append(np.sum(np.isin(X_train.columns.values[:int(0.33*n_features)],X_train.columns.values[processed_shaps_df[processed_shaps_df.p_value<i/100].index.values.astype(np.int32)])) )
    found_noise_features.append(np.sum(1-np.isin(X_train.columns.values[processed_shaps_df[processed_shaps_df.p_value<i/100].index.values.astype(np.int32)],X_train.columns.values[:int(0.33*n_features)])))


In [ ]:
t = powerSHAP_statistical_analysis(
                        shaps_df[:27],
                        0.01,
                        0.99,
                        include_all=False,
    )
t_inf = np.sum(np.isin(X_train.columns.values[:int(0.33*n_features)],X_train.columns.values[t[t.p_value<0.01].index.values.astype(np.int32)])) 
t_noise = np.sum(1-np.isin(X_train.columns.values[t[t.p_value<0.01].index.values.astype(np.int32)],X_train.columns.values[:int(0.33*n_features)]))

In [ ]:
f,axs=plt.subplots(1,1,figsize=(6,5))
axs.plot(range(1,100),np.array(found_features),label="All outputted features",alpha=0.5)
axs.plot(range(1,100),np.array(found_informative_features),label="Informative features found")
axs.plot(range(1,100),np.repeat([83],100-1),label="True Informative features",color='grey',alpha=0.5)
axs.plot([26.99,27],[0,140],color="red",label="Automatic mode",alpha=0.5)
axs.plot(range(1,100),np.array(found_noise_features),label="Noise features")
axs.set_ylabel("Found features")
axs.set_xlabel("Powershap Iterations")
axs.legend()
#axs.set_ylim([0,140])
#axs[2].plot(range(2,200),np.array(found_noise_features))
#axs[2].scatter(27,t_noise,color="red")
#axs[2].set_ylim([0,110])
#axs[2].set_ylabel("Found noise features")
#axs[2].set_xlabel("Iterations")
#f.savefig("logisticregressioncv_iterations_analysis.jpg", bbox_inches="tight")
#f.savefig("logisticregressioncv_iterations_analysis.pdf", bbox_inches="tight")
#f.savefig("logisticregressioncv_iterations_analysis.svg", bbox_inches="tight")
#f.savefig("logisticregressioncv_iterations_analysis.png", bbox_inches="tight")

In [ ]:
selector.transform(X_test)

## sklearn pipeline test

In [ ]:
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier

pipe = Pipeline(
    [
        (
            "selector",
            PowerSHAP(
                CatBoostClassifier(n_estimators=250,verbose=False,use_best_model=True), automatic=True, limit_automatic=100,#power_alpha=0.001,power_req_iterations=0.999,
                #CatBoostRegressor(n_estimators=250,verbose=False), automatic=True, limit_automatic=100,
            ),
        ),
        ("catboost", KNeighborsClassifier()),#(n_estimators=250,verbose=False)),
        #("catboost", CatBoostRegressor(n_estimators=250,verbose=False)),
    ]
)

pipe.fit(X_train, y_train)


from sklearn.metrics import accuracy_score,r2_score


print("Baseline", accuracy_score(KNeighborsClassifier().fit(X_train, y_train).predict(X_test), y_test))
#print("Baseline", r2_score(LinearRegression.fit(X_train, y_train).predict(X_test), y_test))


print("PowerShap feature selection:", accuracy_score(pipe.predict(X_test), y_test))
#print("PowerShap feature selection:", r2_score(pipe.predict(X_test), y_test))



In [ ]:
print("Baseline", accuracy_score(CatBoostClassifier(verbose=False,n_estimators=250).fit(X_train, y_train).predict(X_test), y_test))

In [ ]:
processed_shaps_df = pipe[0]._processed_shaps_df

In [ ]:
len(processed_shaps_df[processed_shaps_df.p_value<0.01])

# benchmarking

In [2]:
def benchmark_dict_print(output_dict):
    for key in output_dict:
        #samples
        print(key+": ")
        for key_inner in output_dict[key]:
            #features
            print(key_inner+": ")
            
            #per informative
            time_str = ""
            
            i = 0
            for avg_time in output_dict[key][key_inner]["Average time"]:
                
                i = i+1
                
                if i ==len(output_dict[key][key_inner]["Average time"]):
                    time_str = time_str + str(avg_time)+"s | "
                else:
                    time_str = time_str + str(avg_time)+"s - "
                  
            key_strs = ""
            for key_inner_inner in output_dict[key][key_inner]: 
                if key_inner_inner != 'Average time':
                    key_strs = key_strs + str(output_dict[key][key_inner][key_inner_inner]["found_informative_features"]) + " ("+str(output_dict[key][key_inner][key_inner_inner]["outputted_noise_features"])+") / "+str(output_dict[key][key_inner][key_inner_inner]["informative_features"]) + " | "
                
            print(time_str+key_strs)
                 

def output_dict_to_df(output_dict):
    df = pd.DataFrame(columns=["n_samples","total_features","informative_features","time","seed","found_informative_features","outputted_noise_features"])
    for samples in output_dict.keys():
        for features in output_dict[samples].keys():
            i = 0
            for informative in output_dict[samples][features].keys():
                if informative != 'Average time':
                    temp = pd.DataFrame.from_dict(output_dict[samples][features][informative])
                    temp["n_samples"]=int(samples)
                    temp["total_features"]=int(features)
                    temp = temp.reset_index()
                    temp = temp.rename(columns={"index":"seed"})
                    temp["time"]=output_dict[samples][features]['Average time'][i]
                    i = i+1
                    df = df.append(temp) 

    return df

## Estimators = 50 Classification

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
import pprint
#sys.path.append("../powershap")

from powershap import PowerSHAP
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False
estimators = 50#250

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerSHAP(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerSHAP(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
output_dict_to_df(output_dict).to_csv("estimators_50_Classification_output_df.csv",index=False)

## Estimators = 250 Classification

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False
estimators = 250
hypercube = False

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerSHAP(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerSHAP(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
        if hypercube:
            output_dict_to_df(output_dict).to_csv("estimators_250_Classification_output_df.csv",index=False)
        else:
            output_dict_to_df(output_dict).to_csv("estimators_250_Classification_output_df_polytone.csv",index=False)

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False
estimators = 250
hypercube = False

output_dict = {}

for n_samples in [5000]:
    output_dict[str(n_samples)]={}
    for n_features in [50,100]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            for n_redundant in [10,25,33,50]:
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
                
                n_redundant_samples = int((n_features-int(n_informative/100*n_features))*(n_redundant/100))
                
                print("Amount of samples = "+str(n_samples))
                print("Total used features = "+str(n_features))
                print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
                print("Redundant features: "+str(n_redundant_samples)+" ("+str(n_redundant)+"%)")
                print("")
                
                found_features = []
                found_idx_features = []
                times = []
                for random_seed in [0,1,2,3,4]:
                    print("Seed "+str(random_seed))

                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=int(n_informative/100*n_features), n_redundant=n_redundant_samples, n_repeated = 0,shuffle=False,random_state=random_seed)
                    X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                    start_time = time.time()
                    if regression_bool:
                        selector = PowerSHAP(
                            model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                            automatic=True
                        )
                    else:
                        selector = PowerSHAP(
                            model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                            automatic=True
                        )
                    selector.fit(X, y)

                    times.append(time.time() - start_time)

                    processed_shaps_df = selector._processed_shaps_df
                    print(50*"-")

                    found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                    found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)

                found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
                found_redundant = [np.sum(np.isin(X.columns.values[int(n_informative/100*n_features):int(n_informative/100*n_features)+n_redundant_samples],f_list)) for f_list in found_idx_features]
                found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)+n_redundant_samples])) for f_list in found_idx_features]
                print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
                print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found redundant: "+str(found_redundant) + "/"+str(n_redundant_samples))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
                print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")

                average_times.append(np.round(times,2))

                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features

                print(100*"=")

        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False
estimators = 250
hypercube = False

output_dict = {}

for n_samples in [5000]:
    output_dict[str(n_samples)]={}
    for n_features in [50]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            for n_redundant in [10,25,33,50]:
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
                
                n_redundant_samples = int((n_features-int(n_informative/100*n_features))*(n_redundant/100))
                
                print("Amount of samples = "+str(n_samples))
                print("Total used features = "+str(n_features))
                print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
                print("Redundant features: "+str(n_redundant_samples)+" ("+str(n_redundant)+"%)")
                print("")
                
                found_features = []
                found_idx_features = []
                times = []
                for random_seed in [0,1,2,3,4]:
                    print("Seed "+str(random_seed))

                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=int(n_informative/100*n_features), n_redundant=n_redundant_samples, n_repeated = 0,shuffle=False,random_state=random_seed)
                    X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                    start_time = time.time()
                    # if classification is False it is a Regression problem
                    model = CatBoostClassifier(verbose=0, n_estimators=250)
                    selector = BorutaShap(model=model,importance_measure='shap',classification=True)
                    selector.fit(X, y,verbose=False)

                    times.append(time.time() - start_time)

                    subset = selector.Subset()
                    print(50*"-")

                    found_features.append(len(subset.columns))
                    found_idx_features.append(subset.columns)

                found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
                found_redundant = [np.sum(np.isin(X.columns.values[int(n_informative/100*n_features):int(n_informative/100*n_features)+n_redundant_samples],f_list)) for f_list in found_idx_features]
                found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)+n_redundant_samples])) for f_list in found_idx_features]
                print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
                print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found redundant: "+str(found_redundant) + "/"+str(n_redundant_samples))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
                print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")

                average_times.append(np.round(times,2))

                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features

                print(100*"=")

        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")

## Estimators = 500 Classification

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
from catboost import CatBoostClassifier,CatBoostRegressor

regression_bool=False
estimators = 500

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerSHAP(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerSHAP(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
        output_dict_to_df(output_dict).to_csv("estimators_500_Classification_output_df.csv",index=False)

## Estimators = 50 Regression

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
from catboost import CatBoostClassifier,CatBoostRegressor

regression_bool=True
estimators = 50

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerSHAP(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerSHAP(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
        
output_dict_to_df(output_dict).to_csv("estimators_50_Regression_output_df.csv",index=False)

## Estimators = 250 regression

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=True
estimators = 250

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerSHAP(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerSHAP(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
        
    
output_dict_to_df(output_dict).to_csv("estimators_250_Regression_output_df.csv",index=False)

## Estimators = 500 regression

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=True
estimators = 500

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerSHAP(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerSHAP(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
        
    
output_dict_to_df(output_dict).to_csv("estimators_500_Regression_output_df.csv",index=False)

## LogisticRegressionCV

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV,LinearRegression

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False
hypercube = False

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,hypercube=hypercube,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()

                selector = PowerSHAP(
                    model = LogisticRegressionCV(max_iter=1000),
                    automatic=True
                )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
        if hypercube:
            output_dict_to_df(output_dict).to_csv("logisticregressioncv_output_df.csv",index=False)
        else:
            output_dict_to_df(output_dict).to_csv("logisticregressioncv_output_df_polytone.csv",index=False)

## RandomForest

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
from sklearn.ensemble import RandomForestClassifier

regression_bool=False

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()

                selector = PowerSHAP(
                    model = RandomForestClassifier(),
                    automatic=True
                )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
output_dict_to_df(output_dict).to_csv("randomforest_output_df.csv",index=False)

In [ ]:
#Stopped simulation at 5000 samples, 20000 samples takes on average 300s per model! so at least 3000s per seed. 
output_dict_to_df(output_dict).to_csv("randomforest_output_df.csv",index=False)

## BorutaShap

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
from BorutaShap import BorutaShap

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                
                # if classification is False it is a Regression problem
                model = CatBoostClassifier(verbose=0, n_estimators=250)
                selector = BorutaShap(model=model,importance_measure='shap',classification=True)

                selector.fit(X=X, y=y, verbose=False)
                subset = selector.Subset()
                
                times.append(time.time() - start_time)
                print(50*"-")
                
                found_features.append(len(subset.columns))
                found_idx_features.append(subset.columns)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
            
        output_dict_to_df(output_dict).to_csv("250_est_Catboost_borutashap_output_df.csv",index=False)
        
        print(100*"=")

## Shapicant

In [3]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerSHAP
import shapicant

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False

output_dict = {}

for n_samples in [5000]:#1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:#[20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
                X["class"]=y
                X = X.reset_index()


                explainer_type = shap.TreeExplainer
                # if classification is False it is a Regression problem
                model = CatBoostClassifier(verbose=0, n_estimators=250,use_best_model=False)
                selector = shapicant.PandasSelector(model, explainer_type, random_state=42)

                train_idx,val_idx = train_test_split(X["index"].values,test_size=0.2,random_state = 0)

                X_train = X[X["index"].isin(train_idx)].copy(deep=True)[list(X.columns.values[1:-1])]
                X_val = X[X["index"].isin(val_idx)].copy(deep=True)[list(X.columns.values[1:-1])]
                Y_train =  X[X["index"].isin(train_idx)]["class"]

                # Run the feature selection
                # If we provide a validation set, SHAP values are computed on it, otherwise they are computed on the training set
                # We can also provide additional parameters to the underlying estimator's fit method through estimator_params
                selector.fit(X_train, Y_train, X_validation=X_val)

                subset = selector.get_features()
                p_values = selector.p_values_

                np.array(subset)

                times.append(time.time() - start_time)
                print(50*"-")
                
                found_features.append(len(subset))
                found_idx_features.append(subset)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
            
        output_dict_to_df(output_dict).to_csv("shapicant_output_df.csv",index=False)
        
        print(100*"=")

ModuleNotFoundError: No module named 'powershap'